# Solving Dominating Set using ASP

This is a small example to show how you can encode the problem of [Dominating Set](https://en.wikipedia.org/wiki/Dominating_set) into ASP and using an ASP solver to solve the problem.

We will use [clingo](https://potassco.org/clingo/) in Python for this. Have a look at [these instructions](asp.ipynb) to see how you can use clingo from Python.

Let's start off with an empty string, that we will fill with an answer set program.

In [1]:
import clingo

asp_code = "";

## Dominating set

In the problem of *Dominating Set (DS)*, you are given as input a finite (undirected) [graph](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)) $G = (V,E)$, consisting of a set $V$ of *vertices* (or *nodes*) and a set $E$ of *edges*. Each edge $e \in E$ consists of a set $\{v_1,v_2\}$ of exactly two vertices in $V$. In addition, you are given as input a positive integer $k \in \mathbb{N}$.

A *dominating set* of a graph $G = (V,E)$ is a subset $D \subseteq V$ of vertices such that for every vertex $v \in V$ holds that either (1) $v$ is in the set $D$ (i.e., $v \in D$), or (2) there exists some vertex $u \in D$ that is connected with an edge to $v$ (i.e., $u \in D$ and $\{ u,v \} \in E$).

In the problem of dominating set, the question is to decide if there exists dominating set $D \subseteq V$ of the graph $G$ that is of size $k$.

## Encoding graphs in ASP

To encode this problem in ASP, we first show how to encode a given graph in ASP.

Take the following example graph $G = (V,E)$, where $V = \{v_1,v_2,v_3,v_4,v_5,v_6,v_7\}$ and $E = \{ \{v_1,v_2\}, \{v_1,v_3\}, \{v_2,v_3\}, \{v_3,v_4\}, \{v_4,v_5\}, \{v_5,v_6\}, \{v_5,v_7\}, \{v_6,v_7\} \}$. We can encode this in ASP using predicates `vertex/1` and `edge/2` as follows.

In [2]:
asp_code += """
    vertex(v1).
    vertex(v2).
    vertex(v3).
    vertex(v4).
    vertex(v5).
    vertex(v6).
    vertex(v7).
    edge(v1,v2).
    edge(v1,v3).
    edge(v2,v3).
    edge(v3,v4).
    edge(v4,v5).
    edge(v5,v6).
    edge(v5,v7).
    edge(v6,v7).
""";

To make things easier, let's also add a rule that states that whenever `edge(U,V)` is true, also `edge(V,U)` is true. This way, we don't have to worry about the order of the vertices in the edges that we declare.

In [3]:
asp_code += """
    edge(U,V) :- edge(V,U).
""";

## Guessing a subset of $k$ vertices

We will use a guess-and-check approach to model the problem of dominating set. In particular, we will add some rules to the answer set program that will guess a set of $k$ vertices, and then after that we will add some constraints that ensure that the set of vertices that we guessed is a dominating set.

We guess the set of $k$ vertices as follows. We declare a constant `k` (with `k=3` for our example), and then we state that among the values for `V` for which `vertex(V)` is true, there are exactly `k` for which also `chosen(V)` is true.

In [4]:
k = 3;
asp_code += "#const k={}.".format(k)
asp_code += """
    k { chosen(V) : vertex(V) } k.
"""

## Verifying that the set dominates all vertices

We now add rules and constraints expressing that the guessed set dominates all vertices in the graph.

Firstly, we define a predicate `dominated/1` that is true for all vertices `V` that are dominated. We have two rules: the first one expresses that every chosen vertex is dominated, and the second one expresses that every vertex connected with an edge to a chosen vertex is dominated.

Finally, we add a constraint that says that every vertex must be dominated.

In [5]:
asp_code += """
    dominated(V) :- vertex(V), chosen(V).
    dominated(V) :- vertex(V), chosen(U), edge(U,V).
    :- vertex(V), not dominated(V).
"""

## Showing the coloring

Finally, we add a `#show` statement that indicates that we are only interested in the predicate `chosen/1`, and we ask clingo to print out some (say, 10) of the dominating sets of the graph of size $k$.

In [6]:
asp_code += """
    #show chosen/1.
"""

control = clingo.Control();
control.add("base", [], asp_code);
control.ground([("base", [])])

def on_model(model):
    print(model.symbols(shown=True));

control.configuration.solve.models = 10;
answer = control.solve(on_model=on_model)

if answer.satisfiable == True:
    print("The graph has a dominating set of size k={}".format(k));
else:
    print("The graph does not have a dominating set of size k={}".format(k));

[chosen(v1), chosen(v2), chosen(v5)]
[chosen(v1), chosen(v3), chosen(v5)]
[chosen(v2), chosen(v3), chosen(v5)]
[chosen(v1), chosen(v4), chosen(v5)]
[chosen(v2), chosen(v4), chosen(v5)]
[chosen(v3), chosen(v4), chosen(v5)]
[chosen(v1), chosen(v3), chosen(v7)]
[chosen(v2), chosen(v3), chosen(v7)]
[chosen(v1), chosen(v4), chosen(v7)]
[chosen(v2), chosen(v4), chosen(v7)]
The graph has a dominating set of size k=3


## Finding a smallest dominating set

In the ASP encoding above, we gave a specific size $k$ of the dominating set that we were after. What we can also do, instead, is ask for the smallest dominating set for a given graph.

To illustrate this, we start by encoding the example graph again.

In [7]:
asp_code = """
    vertex(v1).
    vertex(v2).
    vertex(v3).
    vertex(v4).
    vertex(v5).
    vertex(v6).
    vertex(v7).
    edge(v1,v2).
    edge(v1,v3).
    edge(v2,v3).
    edge(v3,v4).
    edge(v4,v5).
    edge(v5,v6).
    edge(v5,v7).
    edge(v6,v7).
    edge(U,V) :- edge(V,U).
""";

For our definition of the predicate `dominated/1` and the constraint that all vertices should be dominated, we can do exactly as before.

However, we allow any set of vertices to be selected using `chosen/1`, rather than specifying a specific number of vertices to be selected.

In [8]:
asp_code += """
    { chosen(V) : vertex(V) }.
    
    dominated(V) :- vertex(V), chosen(V).
    dominated(V) :- vertex(V), chosen(U), edge(U,V).
    :- vertex(V), not dominated(V).
""";

Then, in addition, we add a `#minimize` statement, that minimizes the amount of vertices that are chosen.

We also add a rule that counts the number of vertices that are chosen (and captures this using the predicate `size/1`).

In [9]:
asp_code += """
    #minimize { 1,chosen(V) : chosen(V) }.
    size(K) :- K = #count { chosen(V) : chosen(V) }.
    
    #show chosen/1.
    #show size/1.
""";

Finally, we call the ASP solver, and extract the smallest dominating sets that are found (together with their size).

In [10]:
control = clingo.Control();
control.add("base", [], asp_code);
control.ground([("base", [])])

def on_model(model):
    if model.optimality_proven == True:
        dominating_set = [];
        size = 0;
        for atom in model.symbols(shown=True):
            if atom.name == "chosen":
                dominating_set.append(str(atom.arguments[0]))
            if atom.name == "size":
                size = int(str(atom.arguments[0]));
        print("Smallest dominating set (of size {}): {{{}}}".format(size,", ".join(dominating_set)));

control.configuration.solve.opt_mode = "optN";
control.configuration.solve.models = 0;

answer = control.solve(on_model=on_model)

Smallest dominating set (of size 2): {v3, v7}
Smallest dominating set (of size 2): {v3, v6}
Smallest dominating set (of size 2): {v3, v5}
Smallest dominating set (of size 2): {v1, v5}
Smallest dominating set (of size 2): {v2, v5}
